In [1]:
!pip -qqq install  transformers
!pip -qqq install langchain
!pip -qqq install sentence-transformers
!pip -qqq install faiss-gpu
!pip -qqq install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain.document_loaders import  WebBaseLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, LLMChain

In [5]:
import os
from getpass import getpass

HF_TOKEN= getpass("enter hf token: ")
os.environ['HUGGINGFACE_ACCESS_TOKEN']=HF_TOKEN

enter hf token: ··········


In [6]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
WEBSITE="https://blog.google/technology/ai/google-gemini-ai/#sundar-note"

data=WebBaseLoader(WEBSITE)
data.request_per_secobd=1

content=data.aload()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.76it/s]


In [ ]:
content

In [10]:
splitter=RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=20)

chunked_doc=splitter.split_documents(content)

In [11]:
chunked_doc[0]

Document(page_content='Introducing Gemini: Google’s most capable AI model yet\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n        The Keyword\n      \n\n\n\n\n    Introducing Gemini: our largest and most capable AI model', metadata={'source': 'https://blog.google/technology/ai/google-gemini-ai/#sundar-note', 'title': 'Introducing Gemini: Google’s most capable AI model yet', 'description': 'Gemini is our most capable and general model, built to be multimodal and optimized for three different sizes: Ultra, Pro and Nano.', 'language': 'en-us'})

In [12]:
chunked_doc[1]

Document(page_content='Share\n\n\n\n\n\n\nTwitter\n\n\n\n\n\nFacebook\n\n\n\n\n\nLinkedIn\n\n\n\n\n\nMail\n\n\n\n\n\n\nCopy link\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n               Latest stories\n            \n\n\n\n              Product updates\n              \n\n\n\n\n\n\n\n\n\n\n\n\n    Product updates', metadata={'source': 'https://blog.google/technology/ai/google-gemini-ai/#sundar-note', 'title': 'Introducing Gemini: Google’s most capable AI model yet', 'description': 'Gemini is our most capable and general model, built to be multimodal and optimized for three different sizes: Ultra, Pro and Nano.', 'language': 'en-us'})

In [13]:
chunked_doc[2]

Document(page_content='Android, Chrome & Play\n            \n\n\n\n\n\n\n\n\n                  Android\n                  \n                \n\n\n\n                  Chrome', metadata={'source': 'https://blog.google/technology/ai/google-gemini-ai/#sundar-note', 'title': 'Introducing Gemini: Google’s most capable AI model yet', 'description': 'Gemini is our most capable and general model, built to be multimodal and optimized for three different sizes: Ultra, Pro and Nano.', 'language': 'en-us'})

In [14]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

In [16]:
vectordb= Chroma.from_documents(chunked_doc, embeddings)

In [18]:
query="what is gemini?"
search=vectordb.similarity_search(query)

In [19]:
search[0].page_content

'10:25\n\n\n\n\nGemini unlocks new scientific insights'

In [20]:
retriver = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={'k':4}
)

In [21]:
retriver.get_relevant_documents(query)

[Document(page_content='10:25\n\n\n\n\nGemini unlocks new scientific insights', metadata={'description': 'Gemini is our most capable and general model, built to be multimodal and optimized for three different sizes: Ultra, Pro and Nano.', 'language': 'en-us', 'source': 'https://blog.google/technology/ai/google-gemini-ai/#sundar-note', 'title': 'Introducing Gemini: Google’s most capable AI model yet'}),
 Document(page_content='model we’ve ever built.Gemini is the result of large-scale collaborative efforts by teams across Google, including our colleagues at Google Research. It was built from the ground up to be multimodal, which means it can generalize and seamlessly', metadata={'description': 'Gemini is our most capable and general model, built to be multimodal and optimized for three different sizes: Ultra, Pro and Nano.', 'language': 'en-us', 'source': 'https://blog.google/technology/ai/google-gemini-ai/#sundar-note', 'title': 'Introducing Gemini: Google’s most capable AI model yet'}

LOADING ZEPHYR 7B MODEL AND MAKING PROMPT TO AUGMENT THE MODEL

In [23]:
import os
from getpass import getpass

hf_token= getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN']=hf_token


llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

··········


In [33]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [34]:
template="""
<|system|>
You are a AI assitant that read the given context and give honest and harmless response.
your grammar should be perfect and relevant.

</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [39]:
prompt=ChatPromptTemplate.from_template(template)
output_parser=StrOutputParser()

In [40]:
rag_chain=(
     {'context':retriver, 'query':RunnablePassthrough()}
    |prompt
    |llm
    |output_parser
)


In [41]:
response=rag_chain.invoke("what is gemini?")

In [42]:
print(response)

Human: 
<|system|>
You are a AI assitant that read the given context and give honest and harmless response.
your grammar should be perfect and relevant.

</s>
<|user|>
what is gemini?
</s>
<|assistant|>
Gemini is one of the 12 zodiac signs in astrology, represented by the symbol of the twins. It is the third sign of the zodiac and is associated with the period between May 21 and June 20. People born under this sign are said to be communicative, adaptable, and curious.


In [43]:
response=rag_chain.invoke("what is State-of-the-art performance?")
print(response)

Human: 
<|system|>
You are a AI assitant that read the given context and give honest and harmless response.
your grammar should be perfect and relevant.

</s>
<|user|>
what is State-of-the-art performance?
</s>
<|assistant|>
State-of-the-art performance refers to the highest level of performance currently achieved in a particular domain or task. It is the best known or established result in that field, and it serves as a benchmark for other researchers or practitioners to strive for. In other words, it is the current best-known solution or approach for a given problem or task.
